In [1]:
pip install --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org  --upgrade pip

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/15/aa/3f4c7bcee2057a76562a5b33ecbd199be08cdb4443a02e26bd2c3cf6fc39/pip-23.3.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 827.3 kB/s eta 0:00:00:0100:010m
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org  pinecone-client pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 252.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 1.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 2.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 654.3 kB/s eta 0:00:000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 328.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 266.4 kB/s eta 0:00:0000:010:01


In [3]:
from pinecone import Pinecone
import os

In [4]:
# initialize connection to pinecone (get API key at app.pc.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
environment = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

# configure client
pc = Pinecone(api_key=api_key)

In [5]:
os.environ.get('PINECONE_API_KEY') 

'3bd6ebfe-51bc-479b-b529-6bdcf0a37ff5'

In [6]:
pc.list_indexes()

{'indexes': [{'dimension': 64,
              'host': 'prices-9atubxs.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'prices',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [7]:
# List indexes
indexes = pc.list_indexes()
print(indexes[0].name)

prices


In [8]:
# Describe index
pc.describe_index(indexes[0].name)

{'dimension': 64,
 'host': 'prices-9atubxs.svc.gcp-starter.pinecone.io',
 'metric': 'cosine',
 'name': 'prices',
 'spec': {'pod': {'environment': 'gcp-starter',
                  'pod_type': 'starter',
                  'pods': 1,
                  'replicas': 1,
                  'shards': 1}},
 'status': {'ready': True, 'state': 'Ready'}}

In [9]:
#create index
new_index_name = "prices"
dimension = 64
metric = "cosine"

In [10]:
# if exists delete
for index in pc.list_indexes().indexes:
    if index.name == new_index_name:
        pc.delete_index(new_index_name)

In [11]:
from pinecone import PodSpec
pc.create_index(
    name=new_index_name, 
    dimension=dimension, 
    metric=metric, 
    spec=PodSpec(
        environment="gcp-starter",
        pod_type="starter",
        pods=1,
        replicas=1,
        shards=1
))

In [12]:
pc.describe_index(new_index_name)

{'dimension': 64,
 'host': 'prices-9atubxs.svc.gcp-starter.pinecone.io',
 'metric': 'cosine',
 'name': 'prices',
 'spec': {'pod': {'environment': 'gcp-starter',
                  'pod_type': 'starter',
                  'pods': 1,
                  'replicas': 1,
                  'shards': 1}},
 'status': {'ready': True, 'state': 'Ready'}}

In [13]:
index = pc.Index(name=new_index_name)

In [14]:
index

In [15]:
import random
ids = [ 'a', 'b', 'c', 'd']
vectors = [ [random.random() for _ in range(64)] for vec in range(4) ]

In [16]:
ids

['a', 'b', 'c', 'd']

In [17]:
len(vectors)

4

In [18]:
# Bulk Insert some data
# [(id1, vecvtor1), (id2, vector2)]
index.upsert(vectors=zip(ids,vectors))

{'upserted_count': 4}

In [19]:
index.describe_index_stats()

{'dimension': 64,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [20]:
# large amounts of data should be inserted in batches over serveral requests

In [21]:
import itertools

vector_dimension = 64
vector_count = 1000

In [22]:
# (id, vector)
example_data_generator = map (
    lambda i :
        (f'id-{i}', [random.random() for _ in range(vector_dimension)]), range(vector_count)
)

# helper function to split data into batches
def chunks(iterable, batch_size):
    
     it = iter(iterable)
     chunk = tuple(itertools.islice(it, batch_size))
     while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))
	

In [23]:
for chunk in chunks(example_data_generator, 100):
    index.upsert(vectors=chunk)

In [ ]:
with pc.Index(new_index_name, pool_threads=30) as index:
    # Send requests in parallel
    async_results = [
        index.upsert(vectors=ids_vectors_chunk, async_req=True)
        for ids_vectors_chunk in chunks(example_data_generator, batch_size=100)
    ]
    # Wait for and retrieve responses (this raises in case of error)
    [async_result.get() for async_result in async_results]